## Aim of the project Loan Prediction Based on Customer Behavior using ML Algorithms 

## Import the libraries and Tools required for this project 

## Create the IAM role to access the S3 bucket 

In [1]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
##from sagemaker.predictor import csv_serializer

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
my_region = boto3.session.Session().region_name # set the region of the instance

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + xgboost_container + " container for your SageMaker endpoint.")

Matplotlib is building the font cache; this may take a moment.


Success - the MySageMakerInstance is in the ap-south-1 region. You will use the 991648021394.dkr.ecr.ap-south-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


## Creation of Unique S3 bucket 

In [2]:
bucket_name = 'ama-02' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


## The urllib.request module provides functionality for making HTTP requests and working with URLs.

In [3]:
import urllib.request
import pandas as pd
import os

try:
    url = "https://raw.githubusercontent.com/{username}/{repository}/{branch}/{path_to_file}"
    file_url = url.format(
        username="ramakanth1997",
        repository="datascience01",
        branch="master",
        path_to_file="Training%20Data.csv"  # Replace spaces with %20
    )
    urllib.request.urlretrieve(file_url, "Training Data.csv")
    print('Success: downloaded Training Data.csv.')
except Exception as e:
    print('Data load error:', e)

try:
    file_path = os.path.join(os.getcwd(), "Training Data.csv")  # Adjust file path if needed
    model_data = pd.read_csv(file_path, index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error:', e)


Success: downloaded Training Data.csv.
Success: Data loaded into dataframe.


## The data is shuffled using sample(frac=1, random_state=1729) before the split to ensure randomization.

In [4]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)


(176400, 12) (75600, 12)


## The function trains the model, makes predictions, and generates a classification report and confusion matrix plot.

In [5]:
def useModel(model, X_data, y_data):

    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=42)

    model = model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("The Classification report : ")
    print(classification_report(y_test, y_pred, zero_division=0))
    
    # Plotting confusion matrix
    sns.set_palette(sns.color_palette())
    _, ax = plt.subplots(figsize=(6,6))
    ax = sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', annot_kws={"size": 20})  
    labels = ['False', 'True']
    ax.set_xticklabels(labels, fontsize=13);
    ax.set_yticklabels(labels[::-1], fontsize=13);
    ax.set_ylabel('Prediction', fontsize=13);
    ax.set_xlabel('Ground Truth', fontsize=13)
    ax.set_title("Confussion - Matix")

##  DataFrame with the remaining columns (excluding specified columns) and saves the resulting DataFrame to a CSV file named 'train.csv' without including the index or header.
## The boto3 library to upload the 'train.csv' file to the specified S3 bucket and prefix location.
## creates a TrainingInput object named s3_input_train that represents the input data for training in Amazon SageMaker. It specifies the S3 location (s3_data) where the training data is stored, and the content type (content_type) of the data in csv.

In [6]:
pd.concat([train_data['Risk_Flag'], train_data.drop(columns=['CITY','STATE','Married/Single','House_Ownership','Car_Ownership','Profession',], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

## Creation of  sagemaker session 
## Creates an Estimator object for XGBoost

In [7]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(xgboost_container,role, instance_count=1, instance_type='ml.m4.xlarge',output_path='s3://{}/{}/output'.format(bucket_name, prefix),sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='binary:logistic',num_round=100)

## training of model using dataset

In [8]:
xgb.fit({'train': s3_input_train})

INFO:sagemaker:Creating training-job with name: xgboost-2023-07-13-13-46-01-831


2023-07-13 13:46:01 Starting - Starting the training job...
2023-07-13 13:46:26 Starting - Preparing the instances for training......
2023-07-13 13:47:29 Downloading - Downloading input data...
2023-07-13 13:47:54 Training - Downloading the training image......
2023-07-13 13:48:57 Training - Training image download completed. Training in progress..Arguments: train
[2023-07-13:13:49:01:INFO] Running standalone xgboost training.
[2023-07-13:13:49:01:INFO] Path /opt/ml/input/data/validation does not exist!
[2023-07-13:13:49:01:INFO] File size need to be processed in the node: 3.81mb. Available memory size in the node: 8593.3mb
[2023-07-13:13:49:01:INFO] Determined delimiter of CSV input is ','
[13:49:01] S3DistributionType set as FullyReplicated
[13:49:01] 176400x6 matrix with 1058400 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[13:49:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[0]#011train-

##  Predict a future outcome based on given dataset. Creates a SageMaker endpoint that you can access. This step may take a few minutes to complete.

In [9]:
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.t2.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2023-07-13-13-49-44-352
INFO:sagemaker:Creating endpoint-config with name xgboost-2023-07-13-13-49-44-352
INFO:sagemaker:Creating endpoint with name xgboost-2023-07-13-13-49-44-352


----!

## The CSVSerializer class is used to serialize input data in CSV format when making requests to an Amazon SageMaker endpoint. 

In [10]:
from sagemaker.serializers import CSVSerializer

test_data_array = test_data.drop(columns=['CITY','STATE','Married/Single','House_Ownership','Car_Ownership','Profession'],axis=1).values #load the data into an array
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(75600,)


## Calculates and prints the overall classification rate, and prints the observed and predicted values.

In [11]:
cm = pd.crosstab(index=test_data['House_Ownership'], columns=np.round(predictions_array), rownames=['CITY'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 37.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    35% (1930)    35% (202)
Purchase        65% (3605)     65% (373) 



In [12]:
xgb_predictor.delete_endpoint(delete_endpoint_config=True)

INFO:sagemaker:Deleting endpoint configuration with name: xgboost-2023-07-13-13-49-44-352
INFO:sagemaker:Deleting endpoint with name: xgboost-2023-07-13-13-49-44-352


In [13]:
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'REXPSBRZ2S0NHDQN',
   'HostId': 'uMotFpwVVB3SAKJH9mYGnVwzj94TgAK1hVBwBtBFhp485EJqVFtGmBtVeTfk36shhFEEfyUgPp8=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'uMotFpwVVB3SAKJH9mYGnVwzj94TgAK1hVBwBtBFhp485EJqVFtGmBtVeTfk36shhFEEfyUgPp8=',
    'x-amz-request-id': 'REXPSBRZ2S0NHDQN',
    'date': 'Thu, 13 Jul 2023 13:52:19 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2023-07-11-12-58-01-075/profiler-output/system/incremental/2023071112/1689080340.algo-1.json'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2023-07-11-13-44-20-358/profiler-output/system/incremental/2023071113/1689083160.algo-1.json'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2023-07-11-13-32-32-262/profiler-output/system/incremental/2023071113/1689082500.algo-1.json'},
   {'Key': '